# Climate Change Risk Dashboard (Realtime)

This dashboard is an example of highlighting risks due to climate change. In this case we study the mainland US and explore risk as a function of global heating, over the period of 1860 - 2100, and comparing the climatic future scenarios A1B and E1.

A measure of human vulnerability and risk is also synthesized and shown as points over the centres of each state. Vulnerability to the hazard (global heating) is shown by the brightness of the point, and the risk the hazard poses is shown by the size of the point.

This version of the dashboard calculates statistics (in-state average temperature and risk metric; anomaly period means _are_ pre-calculated due to their constant nature) in realtime, rather than being fully reliant on pre-computed data to populate the dashboard.

In [ ]:
import os
import warnings

from bokeh.models import ColumnDataSource

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader

import colorcet as cc
import geoviews.feature as gf
import hvplot.pandas
import panel as pn
import param

import iris

import numpy as np

import geopandas as gpd
import pandas as pd

import shapecutter

In [ ]:
pn.extension()

## Load and pre-process shapefile data

In [ ]:
# Load US states shapefile data.
us_states_name = shpreader.natural_earth(
    category="cultural",
    name="admin_1_states_provinces")

us_states = gpd.read_file(us_states_name)

# We only need to keep a few columns of data from the US states geodataframe (gdf).
gdf_drop_cols = list(set(us_states.columns) - set(["name", 'gn_a1_code', "geometry"]))

us_states = us_states.drop(columns=gdf_drop_cols)

In [ ]:
# Add columns for the area and the centre of the geometry.
us_states["area"] = us_states.apply(lambda r: r["geometry"].area, axis=1)
# We do these two separately for the sake of simplicity.
us_states["latitude"] = us_states.apply(lambda r: r["geometry"].centroid.y, axis=1, result_type="expand")
us_states["longitude"] = us_states.apply(lambda r: r["geometry"].centroid.x, axis=1, result_type="expand")

## Climate data

Load the climate data as Iris cubes and provide processing functions for this data.

In [ ]:
a1b_cube = iris.load_cube(iris.sample_data_path("a1b_north_america.nc"))
e1_cube = iris.load_cube(iris.sample_data_path("e1_north_america.nc"))

In [ ]:
def calculate_one_mean(state_code, rcp, year):
    """
    Calculate one mean value, for a specific combination of US state (defined by its state code),
    RCP scenario and year.
    
    """
    cube = a1b_cube if rcp.lower() == "a1b" else e1_cube
    cstr = iris.Constraint(time=lambda cell: cell.point.year == int(year))
    year_cube = cube.extract(cstr)
    geometry = us_states[us_states["gn_a1_code"] == state_code]["geometry"]
    state_year_cube = shapecutter.cut_cube_to_shape(year_cube, geometry)
    if state_year_cube is not None:
        collapsed_cube = state_year_cube.collapsed(["latitude", "longitude"], iris.analysis.MEAN)
        result = float(collapsed_cube.data)
    else:
        result = None
    return result

## Interactive and Realtime Plot

Produce an interactive plot that calculates mean temperatures within states for a selected year and RCP scenario in an on-demand fashion.

### Basic Interface

This interface works (🎉) but it's a bit clunky - notably casting the polygons into a holoviews Pane loses the extent that's otherwise set tight to the limits of the polygons being plotted. Also we are not able to show coastlines on this interface, due to type inconsistencies between the coastlines feature and the holoviews Pane, again.

**Note:** commented out so that running the notebook as a Panel app isn't slowed down by this cell executing before the servable app below gets executed.

In [ ]:
# rcp = pn.widgets.Select(value="a1b", options=["a1b", "e1"], name="RCP")
# year = pn.widgets.IntSlider(value=2021, start=1860, end=2099, step=1, name="Year")
# button = pn.widgets.Button(name="Update", button_type="primary")

# def plot(cdim="none"):
#     return us_states.hvplot.polygons(geo=True,
#                                      c=cdim).opts(
#             toolbar="above", clim=(255, 310),
#             projection=ccrs.LambertConformal(),
#             cmap=cc.cm.CET_L4, colorbar=True)

# def _update(new_ind, rcp, year):
#     if new_ind not in us_states.columns:
#         with warnings.catch_warnings():
#             warnings.simplefilter("ignore")
#             us_states[new_ind] = us_states.apply(
#                 lambda r: calculate_one_mean(r["gn_a1_code"], rcp, year),
#                 axis=1)

# default = f"({rcp.value},{year.value})"
# _update(default, rcp.value, year.value)
# gv_pane = pn.pane.HoloViews(plot(default))

# def update(event):
#     new_ind = f"({rcp.value},{year.value})"
#     with pn.param.set_values(gv_pane, loading=True):
#         _update(new_ind, rcp.value, year.value)
#         gv_pane.object = plot(cdim=new_ind)

# button.on_click(update)

# pn.Row(pn.Column(rcp, year, button), gv_pane)

## Full interface

This takes the `ClimateRiskInterface` from the non-realtime dashboard, strips out extra calculations such as risk calculation, and adds a simple method that calculates mean temperature within a state for a given year and RCP in realtime. We wrap the plot in a `pn.panel` call so that we can put a spinning-wheel loading indicator over the plot while the calculations run in realtime, and so that the interface does not lock up while the calculations execute.

Note that by adding calculated values to the geodataframe `us_states`, we can also use this as a simple cache to avoid recalculating results that have already been calculated.

In [ ]:
class ClimateRiskInterfaceRT(param.Parameterized):
    # Set params that add specific interactivity to the plot.
    rcp = param.ObjectSelector(default="a1b", objects=["a1b", "e1"], label="RCP")
    year = param.Integer(default=2021, bounds=(1860, 2099))
    
    # Set some defaults.
    tmin, tmax = 255, 310
    
    def _update(self, new_ind):
        if new_ind not in us_states.columns:
            with warnings.catch_warnings():  # Explicitly silence Iris collapse warnings.
                warnings.simplefilter("ignore")
                us_states[new_ind] = us_states.apply(
                    lambda r: calculate_one_mean(r["gn_a1_code"], self.rcp, self.year),
                    axis=1)
    
    def prepare_data(self, col_ref):
        """Set up the data to show in the filled polygons."""
        self._update(col_ref)
        return us_states[col_ref]
    
    @param.depends("rcp", "year")
    def plot(self):
        """Plot all the elements of the map."""
        col_ref = f"({self.rcp},{self.year})"
        polyplot = us_states.hvplot.polygons(geo=True,
                                        c=self.prepare_data(col_ref),
                                        hover_cols=["name", col_ref]).opts(
            toolbar="above", clim=(self.tmin, self.tmax),
            projection=ccrs.LambertConformal(),
            cmap=cc.cm.CET_L4, colorbar=True)
            
        result = (gf.coastline * polyplot)
        return result.opts(width=1200, height=800)

In [ ]:
interface = ClimateRiskInterfaceRT()

# button = pn.widgets.Button(name="Update", button_type="primary")
# button.on_click(interface.update)

pn.Row(interface.param, pn.panel(interface.plot, loading_indicator=True)).servable()